In [1]:
import os
import sys
import pickle
import psutil
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

# Define data path
DATA_PATH = "data/"
CHECKPOINT_PATH = "models/"

In [2]:
pids = pickle.load(open(os.path.join(DATA_PATH,'pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'vids.pkl'), 'rb'))
targs = pickle.load(open(os.path.join(DATA_PATH,'targets.pkl'), 'rb'))
prob_targs = pickle.load(open(os.path.join(DATA_PATH, 'prob_targets_allvisits.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'seqs.pkl'), 'rb'))
diags = pickle.load(open(os.path.join(DATA_PATH,'diags.pkl'), 'rb'))
codes = pickle.load(open(os.path.join(DATA_PATH,'icd9.pkl'), 'rb'))
categories = pickle.load(open(os.path.join(DATA_PATH,'categories.pkl'), 'rb'))
sub_categories = pickle.load(open(os.path.join(DATA_PATH,'subcategories.pkl'), 'rb'))
assert len(pids) == len(vids) == len(targs) == len(seqs)

In [3]:
class CustomDataset(Dataset):
    
    def __init__(self, seqs, targets):
        self.x = seqs
        self.y = targets
    
    def __len__(self): 
        return(len(self.x))
    
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

In [4]:
# dataset = CustomDataset(seqs, targs)
dataset = CustomDataset(seqs, prob_targs)

In [5]:
def collate_fn(data):
    """
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
        x_masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
        rev_x: same as x but in reversed time.
        rev_masks: same as x_masks but in reversed time.
        y: a tensor of shape (# patients, max # diagnosis categories) of type torch.float
        y_masks: a tensor of shape (# patients, max # diagnosis categories) of type torch.bool
    """
#     sequences, targets = zip(*data)

#     num_patients = len(sequences)
#     num_visits = [len(patient) for patient in sequences]
#     num_codes = [len(visit) for patient in sequences for visit in patient]
#     batch_num_categories = [len(visit) for patient in targets for visit in patient]

#     max_num_visits = max(num_visits)
#     max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    
#     x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
#     rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     rev_x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
#     for i_patient, patient in enumerate(sequences):   
#         for j_visit, visit in enumerate(patient[:-1]):
#             for k_code, code in enumerate(visit):
#                 x[i_patient, j_visit, k_code] = code
#                 x_masks[i_patient, j_visit, k_code] = 1
#                 if j_visit == len(patient) - 2 and k_code == len(visit) - 1:
#                     rev_visit = x_masks[i_patient].any(dim=1)
#                     rev_x[i_patient, rev_visit] = x[i_patient, rev_visit].flip(0)
#                     rev_x_masks[i_patient, rev_visit] = x_masks[i_patient, rev_visit].flip(0)
                
#     for i_patient, patient in enumerate(targets):   
#         for visit in patient[-1:]:
#             for k_code, code in enumerate(visit):
#                 y[i_patient, k_code] = code
#                 y_masks[i_patient, k_code] = 1
    
#     return x, x_masks, rev_x, rev_x_masks, y, y_masks

    sequences, targets = zip(*data)

    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    batch_num_categories = [len(visit) for patient in targets for visit in patient]
    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    max_num_categories = len(targets[0][0])
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
    y = torch.zeros((num_patients, max_num_categories), dtype=torch.float)
    y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):   
        for j_visit, visit in enumerate(patient[:-1]):
            for k_code, code in enumerate(visit):
                x[i_patient, j_visit, k_code] = code
                x_masks[i_patient, j_visit, k_code] = 1
                if j_visit == len(patient) - 2 and k_code == len(visit) - 1:
                    rev_visit = x_masks[i_patient].any(dim=1)
                    rev_x[i_patient, rev_visit] = x[i_patient, rev_visit].flip(0)
                    rev_x_masks[i_patient, rev_visit] = x_masks[i_patient, rev_visit].flip(0)

#     for i_patient, patient in enumerate(targets):
# #         import pdb; pdb.set_trace()
#         last_visit = patient[-1]
#         y[i_patient,:len(last_visit)] = torch.LongTensor(last_visit)
#         y_masks[i_patient,:len(last_visit)] = torch.BoolTensor(np.ones(len(last_visit)))
    for i_patient, patient in enumerate(targets):
#         import pdb; pdb.set_trace()
        last_visit = patient[-1]
        y[i_patient] = torch.FloatTensor(last_visit)
        y_masks[i_patient] = torch.BoolTensor(np.ones(max_num_categories))
    
    return x, x_masks, rev_x, rev_x_masks, y, y_masks

In [6]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)
val_split = int(len(dataset)*0.10)

In [7]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)

lengths = [train_split, test_split, len(dataset) - (train_split + test_split)]
train_dataset, test_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6561
Length of test dataset: 1312
Length of val dataset: 875


In [8]:
def load_data(train_dataset, test_dataset, val_dataset, collate_fn):
    '''
    Arguments:
        train dataset: train dataset of type `CustomDataset`
        test dataset: test dataset of type `CustomDataset`
        val dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, test_loader, val_loader: train, test and validation dataloaders
    '''
    batch_size = 100
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=collate_fn,
                                               shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           collate_fn=collate_fn,
                                           shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=collate_fn,
                                             shuffle=False)
    
    return train_loader, test_loader, val_loader


train_loader, test_loader, val_loader = load_data(train_dataset, test_dataset, val_dataset, collate_fn)

In [9]:
def sum_embeddings_with_mask(x, masks):
    """
    Arguments:
        x: the embeddings of diagnosis sequence of shape (batch_size, # visits, # diagnosis codes,
            embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, # visits, embedding_dim)
    """
    x[~masks] = 0
    return x.sum(2)

In [10]:
def attention_sum(alpha, v, masks):
    """
    Arguments:
        alpha: the alpha attention weights of shape (batch_size, # visits, 1)
        v: the concatentaion of forward and reversed visit embeddings
            of shape (batch_size, # visits, 2 * embedding_dim)
        masks: the concatentaion of forward and reversed padding masks
            of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        c: the context vector of shape (batch_size, 2 * embedding_dim)
    """
    masks = masks[:,:].any(dim=2)
    v_masked = v.clone()
    v_masked[~masks] = 0
    c = torch.sum(alpha * v_masked, dim=1)
    return c

In [11]:
class AlphaAttention(torch.nn.Module):

    def __init__(self, hidden_dim):
        super().__init__()
        """ 
        Arguments:
            hidden_dim: the hidden dimension
        """
        
        self.a_att = nn.Linear(hidden_dim, 1)

    def forward(self, g):
        """
        Arguments:
            g: the concatenated output tensors both forward and backward from RNN-alpha 
               of shape (batch_size, # visits, 2 * hidden_dim) 
        
        Outputs:
            alpha: the corresponding attention weights of shape (batch_size, # visits, 1)
        """
        m = nn.Softmax(dim=1)
        return m(self.a_att(g))

In [12]:
class BaselineDipole(nn.Module):
    
    def __init__(self, num_codes, num_categories):
        super().__init__()
        """
        Arguments:
            num_codes: total number of diagnosis codes
            num_categories: total number of diagnosis categories to predict
        """
        
        self.embedding = nn.Embedding(num_codes, embedding_dim=180)
        self.rnn = nn.GRU(180, hidden_size=128, batch_first=True)
        self.rev_rnn = nn.GRU(180, hidden_size=128, batch_first=True)
        self.att_a = AlphaAttention(256)
        self.fc = nn.Linear(360, num_categories)
    
    def forward(self, x, masks, rev_x, rev_masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)
            rev_x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            rev_masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

        Outputs:
            logits: logits of shape (batch_size, # categories)
        """
        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.rnn(x)
        rev_x = self.embedding(rev_x)
        rev_x = sum_embeddings_with_mask(rev_x, rev_masks)
        rev_output, _ = self.rev_rnn(rev_x)
        output_cat = torch.cat([output, rev_output], -1)
        x_cat = torch.cat([x, rev_x], -1)
        masks_cat = torch.cat([masks, rev_masks], -1)
        alpha = self.att_a(output_cat)
        c = attention_sum(alpha, x_cat, masks_cat)
        logits = self.fc(c)
        return logits
    

# load the model here
baseline_dipole = BaselineDipole(num_codes = len(codes), num_categories=len(sub_categories))
baseline_dipole

BaselineDipole(
  (embedding): Embedding(4903, 180)
  (rnn): GRU(180, 128, batch_first=True)
  (rev_rnn): GRU(180, 128, batch_first=True)
  (att_a): AlphaAttention(
    (a_att): Linear(in_features=256, out_features=1, bias=True)
  )
  (fc): Linear(in_features=360, out_features=184, bias=True)
)

In [13]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(naive_rnn.parameters(), lr=0.001)
optimizer = torch.optim.Adadelta(baseline_dipole.parameters(), weight_decay=0.001)

In [20]:
def train(model, train_loader, test_loader, n_epochs):
    """
    Arguments:
        model: the BaselineDipole model
        train_loader: training dataloder
        test_loader: validation dataloader
        n_epochs: total number of epochs
    """
#     max_cpu, max_ram = print_cpu_usage()
#     for epoch in range(n_epochs):
#         model.train()
#         train_loss = 0
#         for x, x_masks, rev_x, rev_x_masks, y, y_masks in train_loader:
#             y_hat = model(x, x_masks, rev_x, rev_x_masks) 
#             y_mh = indices_to_multihot(y, y_masks, y_hat)
#             loss = criterion(y_hat, y_mh)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()
#         train_loss = train_loss / len(train_loader)
#         cpu, ram = print_cpu_usage()
#         max_cpu = cpu if cpu > max_cpu else max_cpu
#         max_ram = ram if ram > max_ram else max_ram
#         print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
#         for k in range(5, 31, 5):
#             precision_k, accuracy_k = eval_model(model, test_loader, k=k)
#             print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
#     final_cpu, final_ram = print_cpu_usage()
#     print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")

    max_cpu, max_ram = print_cpu_usage()
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, x_masks, rev_x, rev_x_masks, y, y_masks in train_loader:
            y_hat = model(x, x_masks, rev_x, rev_x_masks)                  
#             y_mh = indices_to_multihot(y, y_masks, y_hat)
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        cpu, ram = print_cpu_usage()
        max_cpu = cpu if cpu > max_cpu else max_cpu
        max_ram = ram if ram > max_ram else max_ram
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        for k in range(5, 31, 5):
            precision_k, accuracy_k = eval_model(model, val_loader, k=k)
            print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
    final_cpu, final_ram = print_cpu_usage()
    print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")

In [21]:
def eval_model(model, test_loader, k=15, n=-1):
    """
    Arguments:
        model: the BaselineDipole model
        test_loader: validation dataloader
        k: value for top k predictions
        n: num of records to evaluate in the batch, value -1 evaulates all records
        
    Outputs:
        precision_k: visit-level precison@k
        accuracy_k: code-level accuracy@k
    """
#     y_pred = torch.LongTensor()
#     y_score = torch.Tensor()
#     y_true = torch.LongTensor()
#     all_precision = []
#     all_accuracy = []
    
#     model.eval()
#     with torch.no_grad():
#         for x, x_masks, rev_x, rev_x_masks, y, y_masks in test_loader:
#             n_eval = y.shape[0] - 1 if n == -1 else n
#             y_hat = model(x, x_masks, rev_x, rev_x_masks)
#             y_hat = F.softmax(y_hat, dim=-1)
#             y_multihot = indices_to_multihot(y, y_masks, y_hat)
#             k_correct = 0
#             total_precision = 0
#             total_accuracy = 0
#             for i in range(n_eval):
#                 visit_correct = 0
#                 y_true = y[i, y_masks[i]]
#                 _, y_pred = torch.topk(y_hat[i], k)

#                 for v in y_true:
#                     if v in y_pred:
#                         visit_correct += 1
#                 visit_precision = visit_correct / min(k, len(y_true))
#                 visit_accuracy = visit_correct / len(y_true)
#                 k_correct += visit_correct
#                 total_precision += visit_precision
#                 total_accuracy += visit_accuracy

#             precision_k = total_precision / n_eval
#             accuracy_k = total_accuracy / n_eval
#             all_precision.append(precision_k)
#             all_accuracy.append(accuracy_k)

#     total_precision_k = np.mean(all_precision)
#     total_accuracy_k = np.mean(all_accuracy)
#     return total_precision_k, total_accuracy_k

    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    
    model.eval()
    with torch.no_grad():
        for x, x_masks, rev_x, rev_x_masks, y, y_masks in test_loader:
            n_eval = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, x_masks, rev_x, rev_x_masks)
            y_hat = F.softmax(y_hat, dim=-1)
            #y_multihot = indices_to_multihot(y, y_masks, y_hat)
            nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            k_correct = 0
            total_precision = 0
            total_accuracy = 0
            for i in range(n_eval):
                visit_correct = 0
                #y_true = y[i, y_masks[i]]
                y_true = nz_cols[nz_rows == i]
                _, y_pred = torch.topk(y_hat[i], k)
                for v in y_true:
                    if v in y_pred:
                        visit_correct += 1
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                k_correct += visit_correct
                total_precision += visit_precision
                total_accuracy += visit_accuracy
            precision_k = total_precision / n_eval
            accuracy_k = total_accuracy / n_eval
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [22]:
def indices_to_multihot(indices, masks, y_hat):
    multihot = torch.zeros_like(y_hat, dtype=torch.float)
    for idx, row in enumerate(indices):
        y_idx = row[masks[idx]].unique()
        multihot[idx] = F.one_hot(y_idx, y_hat.shape[1]).sum(0).float()
    return multihot

In [23]:
def print_cpu_usage():
    load = psutil.getloadavg()[2]
    cpu_usage = (load/os.cpu_count()) * 100
    ram = psutil.virtual_memory()[2]
    print(f"CPU: {cpu_usage:0.2f}")
    print(f"RAM %: {ram}")
    return cpu_usage, ram

In [24]:
n_epochs = 100
%time train(baseline_dipole, train_loader, val_loader, n_epochs)

CPU: 15.14
RAM %: 61.7
CPU: 15.36
RAM %: 61.8
Epoch: 1 	 Training Loss: 4.226230
Epoch: 1 	 Validation precision@k5: 0.6127, accuracy@k5: 0.3357
Epoch: 1 	 Validation precision@k10: 0.5691, accuracy@k10: 0.4914
Epoch: 1 	 Validation precision@k15: 0.6145, accuracy@k15: 0.5957
Epoch: 1 	 Validation precision@k20: 0.6743, accuracy@k20: 0.6714
Epoch: 1 	 Validation precision@k25: 0.7320, accuracy@k25: 0.7319
Epoch: 1 	 Validation precision@k30: 0.7761, accuracy@k30: 0.7761
CPU: 15.78
RAM %: 61.8
Epoch: 2 	 Training Loss: 3.725769
Epoch: 2 	 Validation precision@k5: 0.6346, accuracy@k5: 0.3498
Epoch: 2 	 Validation precision@k10: 0.5952, accuracy@k10: 0.5157
Epoch: 2 	 Validation precision@k15: 0.6388, accuracy@k15: 0.6202
Epoch: 2 	 Validation precision@k20: 0.7030, accuracy@k20: 0.7001
Epoch: 2 	 Validation precision@k25: 0.7596, accuracy@k25: 0.7595
Epoch: 2 	 Validation precision@k30: 0.8032, accuracy@k30: 0.8032
CPU: 16.04
RAM %: 61.8
Epoch: 3 	 Training Loss: 3.620704
Epoch: 3 	 Vali

CPU: 18.37
RAM %: 62.2
Epoch: 19 	 Training Loss: 3.373660
Epoch: 19 	 Validation precision@k5: 0.6652, accuracy@k5: 0.3659
Epoch: 19 	 Validation precision@k10: 0.6199, accuracy@k10: 0.5380
Epoch: 19 	 Validation precision@k15: 0.6696, accuracy@k15: 0.6507
Epoch: 19 	 Validation precision@k20: 0.7330, accuracy@k20: 0.7300
Epoch: 19 	 Validation precision@k25: 0.7848, accuracy@k25: 0.7847
Epoch: 19 	 Validation precision@k30: 0.8227, accuracy@k30: 0.8227
CPU: 18.70
RAM %: 62.2
Epoch: 20 	 Training Loss: 3.370013
Epoch: 20 	 Validation precision@k5: 0.6685, accuracy@k5: 0.3674
Epoch: 20 	 Validation precision@k10: 0.6259, accuracy@k10: 0.5434
Epoch: 20 	 Validation precision@k15: 0.6730, accuracy@k15: 0.6536
Epoch: 20 	 Validation precision@k20: 0.7347, accuracy@k20: 0.7317
Epoch: 20 	 Validation precision@k25: 0.7869, accuracy@k25: 0.7867
Epoch: 20 	 Validation precision@k30: 0.8251, accuracy@k30: 0.8251
CPU: 19.06
RAM %: 62.2
Epoch: 21 	 Training Loss: 3.363741
Epoch: 21 	 Validation 

Epoch: 36 	 Validation precision@k30: 0.8438, accuracy@k30: 0.8438
CPU: 22.54
RAM %: 62.1
Epoch: 37 	 Training Loss: 3.309253
Epoch: 37 	 Validation precision@k5: 0.6988, accuracy@k5: 0.3830
Epoch: 37 	 Validation precision@k10: 0.6623, accuracy@k10: 0.5745
Epoch: 37 	 Validation precision@k15: 0.7041, accuracy@k15: 0.6833
Epoch: 37 	 Validation precision@k20: 0.7610, accuracy@k20: 0.7578
Epoch: 37 	 Validation precision@k25: 0.8077, accuracy@k25: 0.8075
Epoch: 37 	 Validation precision@k30: 0.8454, accuracy@k30: 0.8454
CPU: 22.72
RAM %: 62.1
Epoch: 38 	 Training Loss: 3.305482
Epoch: 38 	 Validation precision@k5: 0.7035, accuracy@k5: 0.3854
Epoch: 38 	 Validation precision@k10: 0.6629, accuracy@k10: 0.5748
Epoch: 38 	 Validation precision@k15: 0.7044, accuracy@k15: 0.6836
Epoch: 38 	 Validation precision@k20: 0.7593, accuracy@k20: 0.7560
Epoch: 38 	 Validation precision@k25: 0.8106, accuracy@k25: 0.8105
Epoch: 38 	 Validation precision@k30: 0.8472, accuracy@k30: 0.8472
CPU: 22.57
RAM 

Epoch: 54 	 Validation precision@k25: 0.8142, accuracy@k25: 0.8140
Epoch: 54 	 Validation precision@k30: 0.8553, accuracy@k30: 0.8553
CPU: 24.15
RAM %: 62.2
Epoch: 55 	 Training Loss: 3.284887
Epoch: 55 	 Validation precision@k5: 0.7070, accuracy@k5: 0.3889
Epoch: 55 	 Validation precision@k10: 0.6751, accuracy@k10: 0.5850
Epoch: 55 	 Validation precision@k15: 0.7180, accuracy@k15: 0.6964
Epoch: 55 	 Validation precision@k20: 0.7683, accuracy@k20: 0.7648
Epoch: 55 	 Validation precision@k25: 0.8137, accuracy@k25: 0.8135
Epoch: 55 	 Validation precision@k30: 0.8533, accuracy@k30: 0.8533
CPU: 24.44
RAM %: 62.3
Epoch: 56 	 Training Loss: 3.285352
Epoch: 56 	 Validation precision@k5: 0.7053, accuracy@k5: 0.3873
Epoch: 56 	 Validation precision@k10: 0.6775, accuracy@k10: 0.5873
Epoch: 56 	 Validation precision@k15: 0.7176, accuracy@k15: 0.6962
Epoch: 56 	 Validation precision@k20: 0.7687, accuracy@k20: 0.7653
Epoch: 56 	 Validation precision@k25: 0.8149, accuracy@k25: 0.8147
Epoch: 56 	 Val

Epoch: 72 	 Validation precision@k20: 0.7654, accuracy@k20: 0.7621
Epoch: 72 	 Validation precision@k25: 0.8158, accuracy@k25: 0.8156
Epoch: 72 	 Validation precision@k30: 0.8558, accuracy@k30: 0.8558
CPU: 27.57
RAM %: 62.1
Epoch: 73 	 Training Loss: 3.277402
Epoch: 73 	 Validation precision@k5: 0.7080, accuracy@k5: 0.3896
Epoch: 73 	 Validation precision@k10: 0.6783, accuracy@k10: 0.5882
Epoch: 73 	 Validation precision@k15: 0.7184, accuracy@k15: 0.6969
Epoch: 73 	 Validation precision@k20: 0.7692, accuracy@k20: 0.7658
Epoch: 73 	 Validation precision@k25: 0.8183, accuracy@k25: 0.8181
Epoch: 73 	 Validation precision@k30: 0.8560, accuracy@k30: 0.8560
CPU: 27.59
RAM %: 62.4
Epoch: 74 	 Training Loss: 3.276973
Epoch: 74 	 Validation precision@k5: 0.7102, accuracy@k5: 0.3899
Epoch: 74 	 Validation precision@k10: 0.6768, accuracy@k10: 0.5869
Epoch: 74 	 Validation precision@k15: 0.7187, accuracy@k15: 0.6974
Epoch: 74 	 Validation precision@k20: 0.7667, accuracy@k20: 0.7633
Epoch: 74 	 Val

Epoch: 90 	 Validation precision@k15: 0.7183, accuracy@k15: 0.6968
Epoch: 90 	 Validation precision@k20: 0.7683, accuracy@k20: 0.7648
Epoch: 90 	 Validation precision@k25: 0.8161, accuracy@k25: 0.8159
Epoch: 90 	 Validation precision@k30: 0.8547, accuracy@k30: 0.8547
CPU: 30.11
RAM %: 62.4
Epoch: 91 	 Training Loss: 3.275612
Epoch: 91 	 Validation precision@k5: 0.7117, accuracy@k5: 0.3908
Epoch: 91 	 Validation precision@k10: 0.6748, accuracy@k10: 0.5846
Epoch: 91 	 Validation precision@k15: 0.7148, accuracy@k15: 0.6936
Epoch: 91 	 Validation precision@k20: 0.7683, accuracy@k20: 0.7649
Epoch: 91 	 Validation precision@k25: 0.8158, accuracy@k25: 0.8156
Epoch: 91 	 Validation precision@k30: 0.8546, accuracy@k30: 0.8546
CPU: 30.38
RAM %: 62.4
Epoch: 92 	 Training Loss: 3.274199
Epoch: 92 	 Validation precision@k5: 0.7119, accuracy@k5: 0.3918
Epoch: 92 	 Validation precision@k10: 0.6756, accuracy@k10: 0.5860
Epoch: 92 	 Validation precision@k15: 0.7164, accuracy@k15: 0.6949
Epoch: 92 	 Val

In [25]:
for k in range(5, 31, 5):
    precision_k, accuracy_k = eval_model(baseline_dipole, test_loader, k=k)
    print(f'Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')

Validation precision@k5: 0.7237, accuracy@k5: 0.3954
Validation precision@k10: 0.6858, accuracy@k10: 0.5859
Validation precision@k15: 0.7164, accuracy@k15: 0.6938
Validation precision@k20: 0.7716, accuracy@k20: 0.7685
Validation precision@k25: 0.8206, accuracy@k25: 0.8204
Validation precision@k30: 0.8533, accuracy@k30: 0.8533


In [26]:
torch.save(baseline_dipole, os.path.join(CHECKPOINT_PATH, "BaselineDipole_100.pth"))